In [1]:
!echo 'tqdm\nPyDrive\npython-levenshtein' > requirements.txt
!pip install -r requirements.txt

In [0]:
import random
import numpy as np
import pandas as pd
import _pickle as pickle
from pprint import pprint

import matplotlib.pyplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

from tqdm import tqdm

import os
from os import listdir
from os.path import isfile, join
import shutil
import stat
import collections
from collections import defaultdict

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import psutil
from tensorflow.python.client import device_lib

In [4]:
uploaded = files.upload()

with open("cookies.txt", 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])

Saving cookies.txt to cookies.txt


In [5]:
!curl -b cookies.txt https://kantiolten-my.sharepoint.com/personal/muriz_serifovic_kantiolten_ch/_layouts/15/download.aspx?SourceUrl=%2Fpersonal%2Fmuriz%5Fserifovic%5Fkantiolten%5Fch%2FDocuments%2FMachine%20Learning%2Fmeta%2Fchefkoch%2Ecsv > chefkoch.csv
!curl -b cookies.txt https://kantiolten-my.sharepoint.com/personal/muriz_serifovic_kantiolten_ch/_layouts/15/download.aspx?SourceUrl=%2Fpersonal%2Fmuriz%5Fserifovic%5Fkantiolten%5Fch%2FDocuments%2FMachine%20Learning%2Fmeta%2Flinks%2Ecsv > links.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  333M  100  333M    0     0  27.7M      0  0:00:12  0:00:12 --:--:-- 38.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 88.1M  100 88.1M    0     0  22.0M      0  0:00:04  0:00:04 --:--:-- 20.2M


In [4]:
import pandas as pd
pic_list = pd.read_csv('links.csv', header=None)
pic_list.head()

,0,1
0,0,https://static.chefkoch-cdn.de/ck.de/rezepte/1...
1,1,https://static.chefkoch-cdn.de/ck.de/rezepte/1...
2,2,https://static.chefkoch-cdn.de/ck.de/rezepte/8...
3,3,https://static.chefkoch-cdn.de/ck.de/rezepte/1...
4,4,https://static.chefkoch-cdn.de/ck.de/rezepte/3...


In [0]:
pic_list = pic_list.drop(pic_list.index[105484])
pic_list = pic_list.drop(pic_list.index[140099])

In [23]:
links = []
for index, row in pic_list.iterrows():
    link = row[1].split(',')
    links.append(link)

        
print(len(links))
count=0
for ll in links:
    for l in ll:
        count+=1
print(count)

190544
880195


In [0]:
import Levenshtein

def similar(a, b):
  return Levenshtein.ratio(a, b)
  
def remove_values_from_list(the_list, val):
  return [value for value in the_list if value[0] != val]

In [0]:
links = remove_values_from_list(links, 'error')

In [26]:
error = []
partial_ids = []
for link in links[:]:
    try:
        partial_ids.append(link[0].split('/')[6]) # extrakt partial id
    except (IndexError):
        error.append(link)
partial_ids[:11]

['111619',
 '174245',
 '8811',
 '149598',
 '32579',
 '187900',
 '58201',
 '301627',
 '306785',
 '80531',
 '22559']

In [55]:
title_ids = []
for link in links:
    try:
        title_ids.append(link[0].split('fix-')[-1].split('.')[0])
    except (IndexError):
        error.append(link)
title_ids[:5]

['melonen-zwieback-brei',
 'johannisbeer-cupcakes-mit-karamellperlen',
 'lamm-aus-dem-ofen',
 'nudelteigtaschen-mit-gemischter-lachsfuelle',
 'kartoffeln-mit-quark']

In [56]:
import csv
def get_list_of_recipes_id():
    recipe_links = []
    chef_file = 'chefkoch.csv'
    with open(chef_file, 'r') as f:
        chefkoch = csv.reader(f)
        for row in chefkoch:
            try:
                recipe_links.append(row[-4])
            except:
                print('ERROR')
                continue 
    return(recipe_links)

all_ids = get_list_of_recipes_id()
all_ids_clean = []
for id in all_ids[1:]: # erste spalte überspringen, es ist der spalten name
    all_ids_clean.append(id[32:].lower()) # 'recipe-' extrahieren
del all_ids[:]
all_ids_clean[:10]

['3449791514066439/bratapfel-rotkohl.html',
 '3449721514056932/fruehstueckswolke-7-aus-fiefhusen.html',
 '3449701514054063/deichgrafensalat-von-der-westkueste.html',
 '3449651514047893/geschichteter-kohl.html',
 '3449681514052511/rinderlendenragout.html',
 '3449481514034645/blaukraut-mit-butter-maroni.html',
 '3449641514047091/sauerbraten-la-tick.html',
 '3449571514042406/punschtorte.html',
 '3449471514033424/gebackene-oberlaender.html',
 '3449241513953107/mcmoes-bunter-pasta-topf-allerlei.html']

In [59]:
print(all_ids_clean[:10])
print(title_ids[:10])
print(partial_ids[:10])

['3449791514066439/bratapfel-rotkohl.html', '3449721514056932/fruehstueckswolke-7-aus-fiefhusen.html', '3449701514054063/deichgrafensalat-von-der-westkueste.html', '3449651514047893/geschichteter-kohl.html', '3449681514052511/rinderlendenragout.html', '3449481514034645/blaukraut-mit-butter-maroni.html', '3449641514047091/sauerbraten-la-tick.html', '3449571514042406/punschtorte.html', '3449471514033424/gebackene-oberlaender.html', '3449241513953107/mcmoes-bunter-pasta-topf-allerlei.html']
['melonen-zwieback-brei', 'johannisbeer-cupcakes-mit-karamellperlen', 'lamm-aus-dem-ofen', 'nudelteigtaschen-mit-gemischter-lachsfuelle', 'kartoffeln-mit-quark', 'mangokuchen-mit-feinen-gewuerzen-nuessen-und-kokosflocken', 'geschnetzeltes', 'dinkel-weizenbrot', 'polenta-nocken-mit-pilzragout', 'mango-creme']
['111619', '174245', '8811', '149598', '32579', '187900', '58201', '301627', '306785', '80531']


In [69]:
import json
matches = {} # key: recipe_id , value: pics_list
for i, n in enumerate(partial_ids):
  match = [ii for ii in all_ids_clean if ii.startswith(n)]
  highest_ratio = 0.0
  favs = []
  for x in match:
    name = x[:-5].split('/')[1]
    ratio = similar(title_ids[i], x)
    if ratio > highest_ratio:
      highest_ratio = ratio
      favs.append(x)
  try:
    r_id = favs[-1].split('/')[0]
    matches[r_id] = links[i]
  except:
    print(match, favs, i, n)
  
  if i % 1000 == 0:
    print(i)
  
  
with open('matches_v3.txt', 'w') as file:
    file.write(json.dumps(matches))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


181000
182000
183000
184000
185000
186000
187000
188000
189000


**match, favs, i, n**

['1712011280048532/fix-kirschgruetze-mit-eiskaffee-sahne.html', '171201074240691/limettenschnitten.html'] [] 26215 171201

['925441197878752/fix-torte.html'] [] 87383 92544

['2065361333924671/fix-und-feierabend-pfanne-mit-gnocchi.html'] [] 130520 206536

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

textfile = drive.CreateFile()
textfile.SetContentFile('matches_v3.txt')
textfile.Upload()